In [1]:
import pandas as pd
import numpy as np

Import Commbank and Amex Expenses

In [2]:
# Commbank csv
commbank_df = pd.read_csv(r'..\00_source_data\CSVData.csv', header=None, names=['tran_date', 'tran_amount','tran_description','balance'])

# Amex csv
amex_df = pd.read_csv(r'..\00_source_data\activity.csv')

Rename Amex columns and add identifiers for both expense source

In [3]:
#rename Amex columns
amex_df = amex_df.rename(columns={'Date':'tran_date', 'Date Processed':'date_processed','Description':'tran_description','Amount':'tran_amount'})

# drop date_processed column from Amex
amex_df = amex_df.drop(columns=['date_processed'], errors='ignore')

# drop balance column from Commbank
commbank_df = commbank_df.drop(columns=['balance'], errors='ignore')

# add expense indicator column:
amex_df['expense_source'] = 'Amex'

commbank_df['expense_source'] = 'Commbank'

Assign type to each transaction in Amex and Commbank

In [ ]:
# np.where is best for one simple if/else but we want to make it a little complex but neat with np select

#amex_df['tran_type'] = np.where(amex_df['tran_amount'] > 0, 'Expense','Income')

# 1. np.select below for Amex (expenses here are in positive):
conditions = [
    amex_df['tran_amount'] > 0,
    amex_df['tran_amount'] < 0,
    amex_df['tran_amount'] ==0
]

choices = ['Expense', 'Income', 'None']

default_value = 'Unknown'

amex_df['tran_type'] = np.select(conditions,choices, default=default_value)

# 2. np.select below for Commbank (expenses here are in negative):
conditions = [
    commbank_df['tran_amount'] > 0,
    commbank_df['tran_amount'] < 0,
    commbank_df['tran_amount'] ==0
]

choices = ['Income', 'Expense', 'None']

default_value = 'Unknown'

commbank_df['tran_type'] = np.select(conditions,choices, default=default_value)

Only keep Expenses from both DFs

In [14]:
# Amex
amex_exp = amex_df['tran_type'] == 'Expense'
amex_df = amex_df.loc[amex_exp]

# Commbank
commbank_exp = commbank_df['tran_type'] == 'Expense'
commbank_df = commbank_df.loc[commbank_exp]

Sort and Union both DFs

In [17]:
amex_df.sort_values('tran_date')
#commbank_df.sort_values('tran_date')

,tran_date,tran_description,tran_amount,expense_source,tran_type
445,1/04/2025,INTEREST CHARGES,5.59,Amex,Expense
443,1/04/2025,8 YOLKS SUCCESS Success,8.91,Amex,Expense
442,1/04/2025,8 YOLKS SUCCESS Success,8.13,Amex,Expense
444,1/04/2025,BP THOMSON LAKES SUCCESS,80.04,Amex,Expense
359,1/05/2025,CHEMIST WAREHOUSE SOUTH SOUTHERN RIVER,20.98,Amex,Expense
...,...,...,...,...,...
202,9/07/2025,KMART SUCCESS 1362 SUCCESS,25.00,Amex,Expense
201,9/07/2025,HAKATA GENSUKE TORI RA NORTHBRIDGE,58.99,Amex,Expense
198,9/07/2025,ASSURANCE VENTURE PTY L SUCCESS,27.70,Amex,Expense
199,9/07/2025,BIG W 0444 GATEWAYS BATEMAM,15.00,Amex,Expense
